In [17]:
#-*-coding:utf-8 -*-

In [1]:
import torch
import tensorflow as tf

from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
#에러 발생시 위 코드 주석으로 대체
#from transformers import WarmupLinearSchedule as get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras_preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import random
import time
import datetime

import csv

In [6]:
def checkGPU():
    n_devices = torch.cuda.device_count()
    print(n_devices)
    for i in range(n_devices):
        print(torch.cuda.get_device_name(i))

In [3]:
import csv
import pandas as pd


def readFile():
    #데이터 읽어오기, csv파일 위치 추가
    chatbot_data = pd.read_csv("CSV 파일 위치")
    #데이터 밸런싱을 신경써서 섞기
    chatbot_data_shuffled = chatbot_data.sample(frace=1).reset_index(drop=True)
    #훈련데이터와 테스트데이터 나누기
    train = chatbot_data_shuffled[:9000]
    test = chatbot_data_shuffled[9000:]
    #문장의 앞마다 [CLS], 문장 종료는 [SEP]를 붙여 인식시킨다.
    #훈련데이터 인식과정
    sentences = ["[CLS] " + str(s) + " [SEP]" for s in train.Q]
    labels = train['label'].values     
    preProcess(senetences, labels, "train")
    #test데이터 인식과정
    sentences = ["[CLS] " + str(s) + " [SEP]" for s in test.Q]
    labels = test['label'].values
    preProcess(senetences, labels, "test")
    

In [6]:
#설명

##################################################
#단어를 토큰으로 만든뒤 정수 인코딩, 패딩 과정 수행
#정수 인코딩 : 텍스트를 숫자로 바꾸는 기법
## -> 각 단어를 고유한 정수에 매핑시키는 작업
# 패딩 과정 : 문장의 길이를 하나로 통일하는 과정
## -> 처리의 간편화를 위함, 하나로 통일할 때 부족한 배열의 값을 0으로 채우는 걸 제로 패딩이라고 함.
## -> 여기서는 최대 128까지로 설정
#기본적으로 자주 등장하는 단어를 단어 집합에 추가
##################################################  
##################################################
## 어텐션이란? 
#쿼리가 주어졌을 때, 이 쿼리와 
#여러 개의 키와의 유사도를 각각 구하고,
#구한 유사도를 가중치로 설정하여 각각의 값을 구한 뒤
#이 값들을 모두 가중합하여 반환하는 함수
######
#단어 벡터에 대해 쿼리, 키 값의 가중치 행렬을 곱해주어
#쿼리, 키, 값 벡터를 얻어낸다.
#그 후 쿼리 벡터에 대해 어텐션 스코어를 구하고,
#이를 이용하여 모든 값 벡터를 가중합하여 어텐션 값을 구하게 된다.
##################################################
## 0 값을 가지는 패딩 토큰에 대해서 
## 어텐션 연산을 불필요하게 수행하지 않도록 단어와 
## 패딩 토큰을 구분할 수 있게 알려주는 것
# 패딩된 데이터가 있을 때 패딩된 값은 0, 패딩되지 않은 단어는 1

In [4]:
##Test set 전처리
import pandas as pd
from transformers import BertTokenizer


def getTokenizer():
        global tokenizer
        tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)

def preProcess(senetences, labels, mode):
    input_ids, attention_masks = tokenizing(sentences)
    
    # 파이토치 텐서로 변환
    if mode == "test": seperateDataTest(input_ids, labels, attention_masks)
    elif mode == "train": seperateDataTrain(input_ids, labels, attention_masks)
    

In [ ]:
def tokenizing(sentences):
    # Word 토크나이저 토큰화
    tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

    # 시퀀스 설정 및 정수 인덱스 변환 & 패딩
    MAX_LEN = 128
    input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
    input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

    # 어텐션 마스크
    attention_masks = []
    for seq in input_ids:
        seq_mask = [float(i>0) for i in seq]
        attention_masks.append(seq_mask)
        
    return input_ids, attention_masks

In [7]:
def seperateDataTest(input_ids, labels, attention_masks):
    test_inputs = torch.tensor(input_ids)
    test_labels = torch.tensor(labels)
    test_masks = torch.tensor(attention_masks)

    # 배치 사이즈 설정 및 데이터 설정
    batch_size = 32
    test_data = TensorDataset(test_inputs, test_masks, test_labels)
    test_sampler = RandomSampler(test_data)
    global test_dataloader
    test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

In [ ]:
def seperateDataTrain(input_ids, labels, attention_masks):
    #훈련셋과 검증셋, 훈련 라벨과 검증 라벨 생성
    #input_ids와 labels를 분리
    train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, labels, random_state= 2000, test_size=0.1)

    #어텐션마스크를 분리
    train_masks, validation_masks, _, _ = train_test_split(attention_masks, input_ids,
                                     random_state=2000,
                                     test_size=0.1)

    #파이토치 텐서로 전환
    train_inputs = torch.tensor(train_inputs)
    train_labels = torch.tensor(train_labels)
    train_masks = torch.tensor(train_masks)

    validation_inputs = torch.tensor(validation_inputs)
    validation_labels = torch.tensor(validation_labels)
    validation_masks = torch.tensor(validation_masks)
    
    #배치사이즈를 설정하고 입력 데이터, 어텐션 마스크, 라벨을 하나의 데이터로 묶어 데이터 생성
    batch_size = 32

    train_data = TensorDataset(train_inputs, train_masks, train_labels)
    train_sampler = RandomSampler(train_data)
    
    global train_dataloader
    train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

    validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
    validation_sampler = SequentialSampler(validation_data)
    
    global validation_dataloader
    validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

In [5]:
#BERT 모델 불러오기 전 GPU 디바이스 설정
def setGPU() :
    if torch.cuda.is_available():    
        device = torch.device("cuda")
        print('There are %d GPU(s) available.' % torch.cuda.device_count())
        print('We will use the GPU:', torch.cuda.get_device_name(0))
    else:
        device = torch.device("cpu")
        print('No GPU available, using the CPU instead.')

There are 1 GPU(s) available.
We will use the GPU: NVIDIA GeForce MX450


In [ ]:
#pretrain된 BERT 모델 불러오기
def getBERT() :
    setGPU()
    model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=2)
    model.cuda()
    #하이퍼 파라미터
    # 옵티마이저
    optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # 학습률(learning rate)
                  eps = 1e-8 
                )

    # 에폭수
    epochs = 4

    # 총 훈련 스텝 : 배치반복 횟수 * 에폭
    total_steps = len(train_dataloader) * epochs

    # 스케줄러 생성
    scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)
    trainModel(model, epochs, optimizer, scheduler)
    testModel(model, epochs, optimizer, scheduler)

In [ ]:
# 정확도 계산 함수
def flatAccuracy(preds, labels):
    
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    return np.sum(pred_flat == labels_flat) / len(labels_flat)
    
    
# 시간 표시 함수
def formatTime(elapsed):

    # 반올림
    elapsed_rounded = int(round((elapsed)))
    
    # hh:mm:ss으로 형태 변경
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:
##모델 학습

#랜덤시드 고정
def trainModel(model, epochs, optimizer, scheduler) :
    seed_val = 42
    random.seed(seed_val)
    np.random.seed(seed_val)
    torch.manual_seed(seed_val)
    torch.cuda.manual_seed_all(seed_val)

    #그래디언트 초기화
    model.zero_grad()

    # 학습
    for epoch_i in range(0, epochs):
    
        # ========================================
        #               Training
        # ========================================
    
        print("")
        print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
        print('Training...')

        # 시작 시간 설정
        t0 = time.time()

        # 로스 초기화
        total_loss = 0

        # 훈련모드로 변경
        model.train()
        
        # 데이터로더에서 배치만큼 반복하여 가져옴
        for step, batch in enumerate(train_dataloader):
            # 경과 정보 표시
            if step % 500 == 0 and not step == 0:
                elapsed = formatTime(time.time() - t0)
                print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

            # 배치를 GPU에 넣음
            batch = tuple(t.to(device) for t in batch)
        
            # 배치에서 데이터 추출
            b_input_ids, b_input_mask, b_labels = batch

            # Forward 수행                
            outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask, 
                        labels=b_labels)
        
            # 로스 구함
            loss = outputs[0]

            # 총 로스 계산
            total_loss += loss.item()

            # Backward 수행으로 그래디언트 계산
            loss.backward()

            # 그래디언트 클리핑
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

            # 그래디언트를 통해 가중치 파라미터 업데이트
            optimizer.step()

            # 스케줄러로 학습률 감소
            scheduler.step()

            # 그래디언트 초기화
            model.zero_grad()

        # 평균 로스 계산
        avg_train_loss = total_loss / len(train_dataloader)            

        print("")
        print("  Average training loss: {0:.2f}".format(avg_train_loss))
        print("  Training epcoh took: {:}".format(formatTime(time.time() - t0)))
        
        # ========================================
        #               Validation
        # ========================================

        print("")
        print("Running Validation...")

        #시작 시간 설정
        t0 = time.time()

        # 평가모드로 변경
        model.eval()

        # 변수 초기화
        eval_loss, eval_accuracy = 0, 0
        nb_eval_steps, nb_eval_examples = 0, 0

        # 데이터로더에서 배치만큼 반복하여 가져옴
        for batch in validation_dataloader:
            # 배치를 GPU에 넣음
            batch = tuple(t.to(device) for t in batch)
        
            # 배치에서 데이터 추출
            b_input_ids, b_input_mask, b_labels = batch
        
            # 그래디언트 계산 안함
            with torch.no_grad():     
                # Forward 수행
                outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        
            # 로스 구함
            logits = outputs[0]

            # CPU로 데이터 이동
            logits = logits.detach().cpu().numpy()
            label_ids = b_labels.to('cpu').numpy()
        
            # 출력 로짓과 라벨을 비교하여 정확도 계산
            tmp_eval_accuracy = flatAccuracy(logits, label_ids)
            eval_accuracy += tmp_eval_accuracy
            nb_eval_steps += 1

        print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
        print("  Validation took: {:}".format(formatTime(time.time() - t0)))

    print("")
    print("Training complete!")

In [ ]:
## 테스트셋 평가

def testModel(model, epochs, optimizer, scheduler):
    #시작 시간 설정
    t0 = time.time()

    # 평가모드로 변경
    model.eval()

    # 변수 초기화
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # 데이터로더에서 배치만큼 반복하여 가져옴
    for step, batch in enumerate(test_dataloader):
        # 경과 정보 표시
        if step % 100 == 0 and not step == 0:
            elapsed = formatTime(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(test_dataloader), elapsed))

        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
    
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch
    
        # 그래디언트 계산 안함
        with torch.no_grad():     
            # Forward 수행
            outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)
    
        # 로스 구함
        logits = outputs[0]

        # CPU로 데이터 이동
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
    
        # 출력 로직과 라벨을 비교하여 정확도 계산
        tmp_eval_accuracy = flatAccuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1

    print("")
    print("Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("Test took: {:}".format(formatTime(time.time() - t0)))

In [ ]:
##새로운 문장 테스트

# 입력 데이터 변환
def convert_input_data(sentences):
    input_ids, attention_masks = tokenizing(sentences)
    # 데이터를 파이토치의 텐서로 변환
    inputs = torch.tensor(input_ids)
    masks = torch.tensor(attention_masks)

    return inputs, masks

# 문장 테스트
def testSentences(sentences):

    # 평가모드로 변경
    model.eval()

    # 문장을 입력 데이터로 변환
    inputs, masks = convert_input_data(sentences)

    # 데이터를 GPU에 넣음
    b_input_ids = inputs.to(device)
    b_input_mask = masks.to(device)
            
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)

    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()

    return logits

In [ ]:
#일상대화 문장 추가
#예시
"""
logits = test_sentences(['더 나은 학교생활 하고 싶어'])
print(logits)

if np.argmax(logits) == 1 :
    print("연애 관련 대화")
elif np.argmax(logits) == 0 :
    print("일상 대화")
"""

In [2]:
def getModel():
    readFile()
    getTokenizer()
    getBERT()

#학습된 모델 가져오기
#getModel()

#새로운 문장 들어왔을 시 치매인지 아닌지 반환
# 1이면 치매, 0이면 일반
def newSentence(S):
    ans = test_sentences([S])
    return np.argmax(ans)

def Test():
    return "TEST"

TEST
